In [1]:
# load full_dataset/data_clean/questions/US/test.jsonl

import json
import os
import pandas as pd

# load data
with open('full_dataset/data_clean/questions/US/test.jsonl') as f:
    data = f.readlines()
data = [json.loads(line) for line in data]

# convert to dataframe
df = pd.DataFrame(data)
df.head()


,question,answer,options,meta_info,answer_idx
0,A junior orthopaedic surgery resident is compl...,Tell the attending that he cannot fail to disc...,{'A': 'Disclose the error to the patient but l...,step1,C
1,A 67-year-old man with transitional cell carci...,Cross-linking of DNA,"{'A': 'Inhibition of thymidine synthesis', 'B'...",step1,E
2,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,"{'A': 'Renal papillary necrosis', 'B': 'Allerg...",step2&3,C
3,A 39-year-old woman is brought to the emergenc...,"Lactose-fermenting, gram-negative rods forming...","{'A': 'Coagulase-positive, gram-positive cocci...",step1,D
4,A 35-year-old man comes to the physician becau...,Ketotifen eye drops,"{'A': 'Erythromycin ointment', 'B': 'Ketotifen...",step2&3,B


In [2]:
# load model
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "ehartford/dolphin-2.2.1-mistral-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
#bnb_config = BitsAndBytesConfig(
#    load_in_8bit=True,
#)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)



/home/san/miniconda3/envs/medqa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.91s/it]


In [3]:
# Re-init the tokenizer so it doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    padding_side="left",
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
sys_prompt = """
You are an extremely skilled doctor, who will accurately respond to medical questions exactly as the user asks.
""".strip()

user_template = """

Please reason generally about answer to the following question:

{question}""".strip()

prompt_template = """<|im_start|>system
 {sys_prompt} <|im_end|>
<|im_start|>user
 {user_prompt} <|im_end|>
<|im_start|>assistant
 """



In [5]:
from ast import literal_eval as le

In [6]:
def options_to_string(options_dict):
    options = []
    for k, v in options_dict.items():
        options.append(f"{k}: {v}")
    return "\n".join(options)

In [7]:
def make_prompt(dataset_row):
    question = dataset_row["question"]
    options = dataset_row["options"]

    # print(dataset_row["answer_idx"])

    options = options_to_string(options)

    question = f"{question}\n{options}"

    prompt = prompt_template.format(
        sys_prompt=sys_prompt,
        user_prompt=user_template.format(question=question)
    )

    return prompt

sample = df.sample(1)
sample_prompt = make_prompt(sample.iloc[0])
print(sample_prompt)

<|im_start|>system
 You are an extremely skilled doctor, who will accurately respond to medical questions exactly as the user asks. <|im_end|>
<|im_start|>user
 Please reason generally about answer to the following question:

A 32-year-old man is brought by ambulance to the emergency room after being involved in a head-on motor vehicle collision at high speed. The patient was found unconscious by the paramedics and regained consciousness briefly during the ambulance ride. Upon arrival at the hospital, the patient’s vitals show: pulse 110/min, respiratory rate 12/min, blood pressure 100/70 mm Hg, and oxygen saturation of 96%. Physical examination reveals an unresponsive man with multiple bruises across the chest and along the upper arms with a laceration on the forehead. His is unresponsive to verbal commands and physical touch. His GCS is 6/15. The right pupil is fixed and dilated. An urgent noncontrast CT of the head is performed and shown in the image. The patient is prepared for eme

In [8]:
from tqdm import tqdm

prompts = []
# reasons = []
outputs = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    prompt = make_prompt(row)
    prompts.append(prompt)
    model_input = eval_tokenizer(prompt, return_tensors="pt").to("cuda")
    output = eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=1024)[0], skip_special_tokens=True)
    outputs.append(output)


  0%|          | 0/160 [00:00<?, ?it/s]/home/san/miniconda3/envs/medqa/lib/python3.10/site-packages/transformers/generation/utils.py:1517: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


256


  0%|          | 0/160 [01:37<?, ?it/s]


KeyboardInterrupt: 

In [23]:

model_input = eval_tokenizer(sample_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=1024)[0], skip_special_tokens=True))

/home/san/miniconda3/envs/medqa/lib/python3.10/site-packages/transformers/generation/utils.py:1517: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<|im_start|> system
 You are an extremely skilled doctor, who will accurately respond to medical questions exactly as the user asks.  
<|im_start|> user
 Please reason the answer to the following question:

A 38-year-old nursing home worker presents to the clinic with complaints of fever, loss of appetite, fatigue, and productive cough for the past couple of months. His fever is low-grade and sputum is often blood-tinged. He has lost 6.8 kg (15.0 lb) during this period and complains of profound night sweats. A plain radiograph of the patient’s chest shows consolidation in the apical part of the right lung. Baseline investigations show the following:
Complete blood count
Hemoglobin 11 g/dL
White blood cells  
Total count 16,000/mm3
Differential count  
Neutrophils 35%
Lymphocytes 54%
Eosinophils 11%
Erythrocyte sedimentation rate 84 mm
The physician suspects that the patient is suffering from a chronic lung infection. Which of the following statements best describes the type of lung inf

In [25]:
from peft import PeftModel

ft_path = (
    "train_outputs/medqa_mistral_dolphin_2_2_1-medqa_mixed_qa_train_20k/checkpoint-750"
)

ft_model = PeftModel.from_pretrained(model, ft_path)

In [27]:

ft_model.eval()
with torch.no_grad():
    reason = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=1024)[0], skip_special_tokens=True)
    print(reason)
    

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<|im_start|> system
 You are an extremely skilled doctor, who will accurately respond to medical questions exactly as the user asks.  
<|im_start|> user
 Please reason the answer to the following question:

A 38-year-old nursing home worker presents to the clinic with complaints of fever, loss of appetite, fatigue, and productive cough for the past couple of months. His fever is low-grade and sputum is often blood-tinged. He has lost 6.8 kg (15.0 lb) during this period and complains of profound night sweats. A plain radiograph of the patient’s chest shows consolidation in the apical part of the right lung. Baseline investigations show the following:
Complete blood count
Hemoglobin 11 g/dL
White blood cells  
Total count 16,000/mm3
Differential count  
Neutrophils 35%
Lymphocytes 54%
Eosinophils 11%
Erythrocyte sedimentation rate 84 mm
The physician suspects that the patient is suffering from a chronic lung infection. Which of the following statements best describes the type of lung inf

In [30]:
follow_up_prompt = f"""
<|im_start|>user
 Please reply only with the option A, B, ... and nothing else <|im_end|>
<|im_start|>assistant
 The letter option for the correct answer is: """


prompt = reason + follow_up_prompt

model_input = eval_tokenizer(prompt, return_tensors="pt").to("cuda")

In [31]:
answer = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=1024)[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


In [33]:
print(answer)

<|im_start|>  system
 You are an extremely skilled doctor, who will accurately respond to medical questions exactly as the user asks.  
<|im_start|>  user
 Please reason the answer to the following question:

A 38-year-old nursing home worker presents to the clinic with complaints of fever, loss of appetite, fatigue, and productive cough for the past couple of months. His fever is low-grade and sputum is often blood-tinged. He has lost 6.8 kg (15.0 lb) during this period and complains of profound night sweats. A plain radiograph of the patient’s chest shows consolidation in the apical part of the right lung. Baseline investigations show the following:
Complete blood count
Hemoglobin 11 g/dL
White blood cells  
Total count 16,000/mm3
Differential count  
Neutrophils 35%
Lymphocytes 54%
Eosinophils 11%
Erythrocyte sedimentation rate 84 mm
The physician suspects that the patient is suffering from a chronic lung infection. Which of the following statements best describes the type of lung i